<a href="https://colab.research.google.com/github/danielreinaux/FeatureEngineeringML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature Engineering:

No nosso projeto, temos duas bases:
* Base de Churn, com informações cadastrais de 1000 pessoas
* Base de Transaições, contendo as transações históricas de compras feitas por essas pessoas, incluindo detalhes como data da compra, valor gasto e categoria do produto

A ideia é estar criando variáveis explicativas, processo fundamental na Engenharia e Ciência de Dados, para melhorar nosso estudo de algoritmos.

### 1. Inicializando o PySpark:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# 2. Informações iniciais

Nesse pontoe estaremos trazendo informações básicas das duas tabelas, e entendendo o número de colunas, o tipo e etc.

Análise fundamental para o começo da criação das variáveis explicativas

In [3]:
df_transacoes = spark.read.csv('/content/base_transacoes.csv', header=True, inferSchema = True)

df_transacoes.createOrReplaceTempView('df_transacoes')

In [4]:
df_publico = spark.read.csv('/content/base_churn.csv', header=True, inferSchema = True)

df_publico.createOrReplaceTempView('df_publico')

### 2.1 Vendo Schemas:
Isso nos possibilita ver as colunas das bases, entendendo também o tipo de cada coluna

In [5]:
df_transacoes.printSchema()

root
 |-- ID Transação: integer (nullable = true)
 |-- ID Cliente: integer (nullable = true)
 |-- Data: string (nullable = true)
 |-- Valor: double (nullable = true)
 |-- Categoria: string (nullable = true)



In [6]:
df_publico.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- Dias desde a Inscrição: integer (nullable = true)
 |-- Usou Suporte: integer (nullable = true)
 |-- Plano: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df_transacoes.show(10, False)

+------------+----------+-----------------------------+------------------+-----------+
|ID Transação|ID Cliente|Data                         |Valor             |Categoria  |
+------------+----------+-----------------------------+------------------+-----------+
|1           |1         |2022-11-25 13:50:26.548672560|57.287427536330505|Esportes   |
|2           |1         |2020-01-19 12:27:36.637168141|97.07199340552512 |Alimentos  |
|3           |1         |2021-12-28 12:33:58.938053096|169.10581012381087|Livros     |
|4           |1         |2022-02-05 01:39:49.380530968|199.38694865538451|Roupas     |
|5           |1         |2022-11-16 23:06:54.159292032|160.00228343317622|Eletrônicos|
|6           |1         |2020-04-25 12:42:28.672566372|9.842481270765422 |Alimentos  |
|7           |1         |2022-10-31 22:05:18.584070800|76.90706330227667 |Eletrônicos|
|8           |1         |2020-10-06 12:14:52.035398228|53.20607404593595 |Roupas     |
|9           |1         |2022-11-02 12:50:5

In [8]:
df_publico.show()

+---+-----+------+----------------------+------------+-------------+-----+
| ID|Idade|Gênero|Dias desde a Inscrição|Usou Suporte|        Plano|Churn|
+---+-----+------+----------------------+------------+-------------+-----+
|  1|   21|     F|                  1331|           1|Intermediário|    1|
|  2|   21|     M|                  1160|           0|Intermediário|    0|
|  3|   62|     M|                   454|           1|       Básico|    0|
|  4|   64|     M|                   226|           1|Intermediário|    0|
|  5|   61|     M|                   474|           1|     Avançado|    0|
|  6|   18|     M|                   419|           0|       Básico|    0|
|  7|   52|     M|                  1334|           0|       Básico|    0|
|  8|   44|     M|                  1124|           1|Intermediário|    0|
|  9|   52|     M|                  1256|           1|Intermediário|    0|
| 10|   64|     F|                  1197|           0|       Básico|    0|
| 11|   50|     F|       

# 3 Análise Inicial

Nessa etapa, estaremos fazendo consultas simples, nos ajudando a trazer contexto as tabelas

### 3.1 Contagem de Transações por Cliente

Veremos quantas transações cada cliente realizou. Isso nos ajudará a identificar clientes mais ativos

In [40]:
df_qtd_transacoes = spark.sql("""
SELECT `ID Cliente`, COUNT(*) as QTD_TRANSACOES
from df_transacoes
group by `ID Cliente`
order by QTD_TRANSACOES DESC
""")

df_qtd_transacoes.createOrReplaceTempView('df_qtd_transacoes')

In [41]:
df_qtd_transacoes.show()

+----------+--------------+
|ID Cliente|QTD_TRANSACOES|
+----------+--------------+
|       959|            19|
|       852|            19|
|       235|            19|
|       715|            19|
|       699|            19|
|       966|            19|
|       120|            19|
|       246|            19|
|       797|            19|
|       689|            19|
|       861|            19|
|       225|            19|
|       111|            19|
|       844|            19|
|       919|            19|
|       822|            19|
|       580|            19|
|       253|            19|
|       950|            19|
|        53|            19|
+----------+--------------+
only showing top 20 rows



### 3.2 Valor Médio das Transações por Clientes

Isso ajudará a entender o ocmportamento de compra dos clientes

In [42]:
vl_med_tras = spark.sql("""
SELECT `ID Cliente`, ROUND(AVG(Valor), 2) as VL_MED_TRANSACOES
from df_transacoes
group by `ID Cliente`
""")

vl_med_tras.createOrReplaceTempView('vl_med_tras')

vl_med_tras.show()

+----------+-----------------+
|ID Cliente|VL_MED_TRANSACOES|
+----------+-----------------+
|       148|            98.22|
|       463|            93.29|
|       471|           135.73|
|       496|            85.31|
|       833|           101.95|
|       243|            98.88|
|       392|            114.6|
|       540|            92.87|
|       623|            93.95|
|       737|           105.95|
|       858|            129.1|
|       897|            88.58|
|        31|           102.14|
|       516|           136.59|
|        85|            65.46|
|       137|           118.51|
|       251|            88.28|
|       451|            114.2|
|       580|           103.75|
|       808|           100.19|
+----------+-----------------+
only showing top 20 rows



### 3.3 Valor médio das transações por categoria

In [43]:
vl_med_tras_cat = spark.sql("""
SELECT Categoria, ROUND(AVG(VALOR), 2) as VL_MED_CATEGORIA
from df_transacoes
group by Categoria
""")

vl_med_tras_cat.createOrReplaceTempView('vl_med_tras_cat')

vl_med_tras_cat.show()

+-----------+----------------+
|  Categoria|VL_MED_CATEGORIA|
+-----------+----------------+
|     Livros|          105.34|
|Eletrônicos|          101.27|
|  Alimentos|          104.43|
|   Esportes|          101.98|
|     Roupas|          100.32|
+-----------+----------------+



#4. Criando Variáveis Explicativas

### 4.1 Data da última Transação:

In [44]:
df_ult_tras = spark.sql("""
SELECT `ID Cliente`, DATEDIFF(current_date(), MAX(DATA)) as ULTIMA_TRANSACAO_EM_DIAS
FROM df_transacoes
group by `ID Cliente`
""")

df_ult_tras.createOrReplaceTempView('df_ult_tras')

df_ult_tras.show()

+----------+------------------------+
|ID Cliente|ULTIMA_TRANSACAO_EM_DIAS|
+----------+------------------------+
|       148|                     569|
|       463|                     594|
|       471|                     557|
|       496|                    1030|
|       833|                     718|
|       243|                     722|
|       392|                     564|
|       540|                     547|
|       623|                     683|
|       737|                     663|
|       858|                    1405|
|       897|                     613|
|        31|                     573|
|       516|                    1446|
|        85|                     554|
|       137|                     561|
|       251|                     801|
|       451|                     608|
|       580|                     582|
|       808|                     558|
+----------+------------------------+
only showing top 20 rows



### 4.2 Frequência de Compra:

Quantidade de compras que o cliente fez, dividida pelo número de meses desde sua inscrição:

In [46]:
df_freq_compra_mensal = spark.sql("""
SELECT `ID Cliente`, (COUNT(*) / (p.`Dias desde a Inscrição`/ 30)) as FREQ_COMPRA_MENSAL
FROM
  df_transacoes as t
LEFT JOIN
  df_publico as p
on
  t.`ID Cliente` = p.`ID`

GROUP BY t.`ID Cliente`, p.`Dias desde a Inscrição`

""")

df_freq_compra_mensal.createOrReplaceTempView('df_freq_compra_mensal')

df_freq_compra_mensal.show()

+----------+--------------------+
|ID Cliente|  FREQ_COMPRA_MENSAL|
+----------+--------------------+
|       594| 0.06569343065693431|
|       327|  0.8042895442359249|
|       627|  0.1748704663212435|
|       692|  2.7127659574468086|
|       204| 0.08995502248875561|
|       728|  0.6060606060606061|
|       361|  0.7090103397341211|
|        57| 0.11335012594458439|
|       283| 0.22751895991332613|
|       615|  1.3255813953488371|
|       709|  0.0494641384995878|
|       900| 0.07042253521126761|
|       926|  0.1868629671574179|
|       942|  1.0169491525423728|
|        14|0.059721300597213006|
|        42|  0.4113924050632911|
|       225|  1.5489130434782608|
|       330| 0.27892561983471076|
|       562| 0.11406844106463877|
|       786|  0.4444444444444444|
+----------+--------------------+
only showing top 20 rows



### 4.3 Total Gasto

Soma de todos os valores gastos pelo cliente em todas as suas transações

In [47]:
df_total_gasto = spark.sql("""
SELECT `ID Cliente`, ROUND(SUM(Valor),2) as VALOR_TOTAL_GASTO
FROM df_transacoes
GROUP BY `ID Cliente`
ORDER BY `VALOR_TOTAL_GASTO` DESC
""")

df_total_gasto.createOrReplaceTempView('df_total_gasto')

df_total_gasto.show()

+----------+-----------------+
|ID Cliente|VALOR_TOTAL_GASTO|
+----------+-----------------+
|       759|          2498.27|
|       910|          2370.01|
|       470|          2361.46|
|       718|          2347.81|
|       983|          2346.24|
|       692|          2329.25|
|       471|          2307.39|
|       407|          2297.79|
|       685|          2283.31|
|       699|          2278.31|
|       668|          2269.66|
|       246|          2261.08|
|       167|          2255.57|
|       162|          2254.72|
|       917|          2232.73|
|       861|          2229.12|
|       480|          2193.45|
|       506|          2185.63|
|       794|           2185.2|
|       630|          2171.44|
+----------+-----------------+
only showing top 20 rows



### 4.4 Categoria Favorita:
Categoria de produto em que o cliente gastou a maior quantia

1. Primeiro faremos o valor gasto em cada uma das Categorias, por pessoa:

In [48]:
df_temp = spark.sql("""
SELECT `ID Cliente`,
    round(sum(case when Categoria = 'Esportes' then Valor else 0 end), 2) as VL_TOT_ESPORTES,
    round(sum(case when Categoria = 'Roupas' then Valor else 0 end), 2) as VL_TOT_ROUPAS,
    round(sum(case when Categoria = 'Livros' then Valor else 0 end), 2) as VL_TOT_LIVROS,
    round(sum(case when Categoria = 'Eletrônicos' then Valor else 0 end), 2) as VL_TOT_ELETRONICOS,
    round(sum(case when Categoria = 'Alimentos' then Valor else 0 end), 2) as VL_TOT_ALIMENTOS
FROM df_transacoes
GROUP BY `ID Cliente`
ORDER BY `ID Cliente`
""")
df_temp.createOrReplaceTempView("df_temp")

In [16]:
df_temp.show()

+----------+---------------+-------------+-------------+------------------+----------------+
|ID Cliente|VL_TOT_ESPORTES|VL_TOT_ROUPAS|VL_TOT_LIVROS|VL_TOT_ELETRONICOS|VL_TOT_ALIMENTOS|
+----------+---------------+-------------+-------------+------------------+----------------+
|         1|         226.73|       450.79|       551.06|            396.72|          351.26|
|         2|         275.47|          0.0|       340.59|            292.16|          109.58|
|         3|         327.09|          0.0|       105.81|            317.38|          247.52|
|         4|         199.35|       367.89|        48.18|            291.16|          103.19|
|         5|         436.01|       303.13|        16.25|            707.41|          292.37|
|         6|         395.05|       499.17|       371.56|            202.33|          373.42|
|         7|          137.2|       227.68|       463.88|            572.15|          270.44|
|         8|         150.17|       118.38|        69.79|            38

Agora faremos a comparação entre cada Categoria, para definir qual a categoria favorita. Para isso, faremos:

* Transformação das Colunas em Linhas, com o 'UNION ALL':
  - Nesse caso, vamos converter valroes de diferentes categorias em linhas separadas para facilitar a comparação

* Classificação e Numeração com 'ROW_NUMBER()'
  - Objetivo: Atribuir um número de linha a cada categoria por cliente, ordenando pelo valor total gasto

In [49]:
df_favorita = spark.sql("""
SELECT `ID Cliente`, Categoria as Categoria_Favorita
FROM (
    SELECT `ID Cliente`, Categoria, Valor_Total,
           ROW_NUMBER() OVER (PARTITION BY `ID Cliente` ORDER BY Valor_Total DESC) as rn
    FROM (
        SELECT `ID Cliente`, 'Esportes' as Categoria, VL_TOT_ESPORTES as Valor_Total FROM df_temp
        UNION ALL
        SELECT `ID Cliente`, 'Roupas' as Categoria, VL_TOT_ROUPAS as Valor_Total FROM df_temp
        UNION ALL
        SELECT `ID Cliente`, 'Livros' as Categoria, VL_TOT_LIVROS as Valor_Total FROM df_temp
        UNION ALL
        SELECT `ID Cliente`, 'Eletrônicos' as Categoria, VL_TOT_ELETRONICOS as Valor_Total FROM df_temp
        UNION ALL
        SELECT `ID Cliente`, 'Alimentos' as Categoria, VL_TOT_ALIMENTOS as Valor_Total FROM df_temp
    ) t
) ranked
WHERE rn = 1
""")

df_favorita.createOrReplaceTempView('df_favorita')


df_favorita.show(truncate=False)


+----------+------------------+
|ID Cliente|Categoria_Favorita|
+----------+------------------+
|148       |Alimentos         |
|463       |Roupas            |
|471       |Roupas            |
|496       |Roupas            |
|833       |Livros            |
|243       |Eletrônicos       |
|392       |Roupas            |
|540       |Esportes          |
|623       |Roupas            |
|737       |Eletrônicos       |
|858       |Roupas            |
|897       |Alimentos         |
|31        |Livros            |
|516       |Roupas            |
|85        |Livros            |
|137       |Roupas            |
|251       |Esportes          |
|451       |Livros            |
|580       |Esportes          |
|808       |Eletrônicos       |
+----------+------------------+
only showing top 20 rows



###4.5 Gasto Médio por Transação:
Toal gasto dividido pelo número total de transações:


In [50]:
df_gasto_medio_tras = spark.sql("""
SELECT `ID Cliente`,
    ROUND(SUM(VALOR) / COUNT(*), 2) as VALOR_TOTAL_POR_TRANS
FROM df_transacoes
GROUP BY `ID Cliente`
""")

df_gasto_medio_tras.createOrReplaceTempView('df_gasto_medio_tras')

df_gasto_medio_tras.show()

+----------+---------------------+
|ID Cliente|VALOR_TOTAL_POR_TRANS|
+----------+---------------------+
|       148|                98.22|
|       463|                93.29|
|       471|               135.73|
|       496|                85.31|
|       833|               101.95|
|       243|                98.88|
|       392|                114.6|
|       540|                92.87|
|       623|                93.95|
|       737|               105.95|
|       858|                129.1|
|       897|                88.58|
|        31|               102.14|
|       516|               136.59|
|        85|                65.46|
|       137|               118.51|
|       251|                88.28|
|       451|                114.2|
|       580|               103.75|
|       808|               100.19|
+----------+---------------------+
only showing top 20 rows



### 4.6 Duração de Assinatura

Números de dias desde que o cliente se inscreceu, até a data mais recente no conjunto de dados.

Para esse, precisaremos de:
- Data em que o cliente se inscreveu (temos na df_publico)
- A última transação do cliente

Pegando a última transação do Cliente:

In [19]:
df_dt_max = spark.sql("""
SELECT `ID Cliente`, date_format(MAX(DATA), 'yyyy-MM-dd HH:mm') as DATA_ULTIMA_TRANSACAO
from df_transacoes
GROUP BY `ID Cliente`
ORDER BY `ID Cliente`
""")

df_dt_max.createOrReplaceTempView("df_dt_max")

In [20]:
df_dt_max.show()

+----------+---------------------+
|ID Cliente|DATA_ULTIMA_TRANSACAO|
+----------+---------------------+
|         1|     2022-12-18 22:35|
|         2|     2021-12-07 02:20|
|         3|     2022-11-21 11:38|
|         4|     2022-12-12 14:07|
|         5|     2022-12-22 06:41|
|         6|     2022-09-06 03:32|
|         7|     2022-09-19 14:32|
|         8|     2022-09-18 10:07|
|         9|     2022-08-30 08:44|
|        10|     2022-10-01 08:12|
|        11|     2022-08-18 09:54|
|        12|     2022-06-02 10:18|
|        13|     2022-10-17 09:14|
|        14|     2021-08-18 12:29|
|        15|     2021-08-26 16:53|
|        16|     2022-10-19 15:30|
|        17|     2022-11-13 09:50|
|        18|     2022-10-21 14:01|
|        19|     2022-06-15 16:08|
|        20|     2022-09-22 22:39|
+----------+---------------------+
only showing top 20 rows



Visto que Dias desde a inscrição considera apenas dias, transformaremos a coluna de Dias desde a inscrição no formato Data.
Para isso, faremos a 'adição' com o dia de hoje

In [21]:
df_publico = spark.sql("""
SELECT ID, `Dias desde a Inscrição`,
       date_add(current_date(), -`Dias desde a Inscrição`) as DATA_INSCRICAO
FROM df_publico
""")
df_publico.createOrReplaceTempView("df_publico")

In [22]:
df_publico.show()

+---+----------------------+--------------+
| ID|Dias desde a Inscrição|DATA_INSCRICAO|
+---+----------------------+--------------+
|  1|                  1331|    2020-11-03|
|  2|                  1160|    2021-04-23|
|  3|                   454|    2023-03-30|
|  4|                   226|    2023-11-13|
|  5|                   474|    2023-03-10|
|  6|                   419|    2023-05-04|
|  7|                  1334|    2020-10-31|
|  8|                  1124|    2021-05-29|
|  9|                  1256|    2021-01-17|
| 10|                  1197|    2021-03-17|
| 11|                  1820|    2019-07-03|
| 12|                   550|    2022-12-24|
| 13|                  1466|    2020-06-21|
| 14|                  1507|    2020-05-11|
| 15|                  1292|    2020-12-12|
| 16|                  1514|    2020-05-04|
| 17|                  1483|    2020-06-04|
| 18|                    32|    2024-05-25|
| 19|                  1203|    2021-03-11|
| 20|                  1157|    

In [23]:
df_intervalo = spark.sql("""
SELECT dm.`ID Cliente`, datediff(dm.DATA_ULTIMA_TRANSACAO, pb.DATA_INSCRICAO) as DURACAO_ASSINATURA
from df_dt_max dm
  INNER JOIN
    df_publico pb

  ON dm.`ID Cliente` = pb.ID
  ORDER BY pb.ID
""")
df_intervalo.createOrReplaceTempView("df_intervalo")

In [24]:
df_intervalo.show()

+----------+------------------+
|ID Cliente|DURACAO_ASSINATURA|
+----------+------------------+
|         1|               775|
|         2|               228|
|         3|              -129|
|         4|              -336|
|         5|               -78|
|         6|              -240|
|         7|               688|
|         8|               477|
|         9|               590|
|        10|               563|
|        11|              1142|
|        12|              -205|
|        13|               848|
|        14|               464|
|        15|               257|
|        16|               898|
|        17|               892|
|        18|              -582|
|        19|               461|
|        20|               514|
+----------+------------------+
only showing top 20 rows



Percebemos que existe uma incongruência nos dados. Alguns ID fizeram transações antes de sequer serem inscritos.

### 4.7 Número de Categorias Compradas:
Quantidade de categorias diferentes das quais o cliente comprou

In [25]:
df_qtd_categorias = spark.sql("""

SELECT `ID Cliente`, COUNT(*) as QTD_CATEGORIAS_COMPRADAS
FROM df_transacoes
GROUP BY `ID Cliente`, Categoria
""")

df_qtd_categorias.createOrReplaceTempView("df_qtd_categorias")

In [26]:
df_qtd_categorias.show()

+----------+------------------------+
|ID Cliente|QTD_CATEGORIAS_COMPRADAS|
+----------+------------------------+
|        73|                       2|
|        87|                       3|
|       130|                       4|
|       161|                       1|
|       193|                       3|
|       235|                       5|
|       330|                       4|
|       375|                       2|
|       386|                       1|
|       408|                       4|
|       472|                       2|
|       610|                       2|
|       630|                       5|
|       634|                       3|
|       682|                       2|
|       706|                       2|
|       721|                       1|
|       849|                       2|
|       851|                       5|
|       951|                       3|
+----------+------------------------+
only showing top 20 rows



# 5 - Criação de Variáveis Históricas:

Análise fundamental e etc...

### 5.1 Média do Valor Gasto nas categorias nos últimos 3 Meses:
Esta variável calcula a média dos gastos do cliente na categoria 'Esportes' nesse período

In [27]:
df_valor_medio_3m = spark.sql("""
SELECT
    `ID Cliente`,
    ROUND(AVG(CASE WHEN Categoria = 'Esportes' THEN Valor ELSE NULL END), 2) AS MD_VLR_ESPORTES,
    ROUND(AVG(CASE WHEN Categoria = 'Alimentos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ALIMENTOS,
    ROUND(AVG(CASE WHEN Categoria = 'Roupas' THEN Valor ELSE NULL END), 2) AS MD_VLR_ROUPAS,
    ROUND(AVG(CASE WHEN Categoria = 'Livros' THEN Valor ELSE NULL END), 2) AS MD_VLR_LIVROS,
    ROUND(AVG(CASE WHEN Categoria = 'Eletrônicos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ELETRONICOS
FROM
    df_transacoes
WHERE
    Data >= DATE_SUB(current_date(), 720)
GROUP BY
    `ID Cliente`
""")

df_valor_medio_3m.createOrReplaceTempView('df_valor_medio_3m')

In [28]:
df_valor_medio_3m.show()

+----------+---------------+----------------+-------------+-------------+------------------+
|ID Cliente|MD_VLR_ESPORTES|MD_VLR_ALIMENTOS|MD_VLR_ROUPAS|MD_VLR_LIVROS|MD_VLR_ELETRONICOS|
+----------+---------------+----------------+-------------+-------------+------------------+
|       148|         105.19|           83.42|       139.53|         null|             63.83|
|       463|          63.06|            null|        80.97|       156.69|              null|
|       471|           null|           86.58|       141.96|         null|              null|
|       833|           null|            null|         null|       116.39|              null|
|       392|         191.38|          187.32|         null|        54.72|              null|
|       540|          59.37|            null|         null|        74.73|              null|
|       623|         109.34|            null|         null|         null|              null|
|       737|           null|            null|         15.5|         nu

### 5.2 Média do Valor Gasto nas categorias nos últimos 6 Meses:


In [54]:
df_valor_medio_6m = spark.sql("""
SELECT
    `ID Cliente`,
    ROUND(AVG(CASE WHEN Categoria = 'Esportes' THEN Valor ELSE NULL END), 2) AS MD_VLR_ESPORTES,
    ROUND(AVG(CASE WHEN Categoria = 'Alimentos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ALIMENTOS,
    ROUND(AVG(CASE WHEN Categoria = 'Roupas' THEN Valor ELSE NULL END), 2) AS MD_VLR_ROUPAS,
    ROUND(AVG(CASE WHEN Categoria = 'Livros' THEN Valor ELSE NULL END), 2) AS MD_VLR_LIVROS,
    ROUND(AVG(CASE WHEN Categoria = 'Eletrônicos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ELETRONICOS
FROM
    df_transacoes
WHERE
    Data >= DATE_SUB(current_date(), 810)
GROUP BY
    `ID Cliente`
""")

df_valor_medio_6m.createOrReplaceTempView('df_valor_medio_6m')

In [55]:
df_valor_medio_6m.show()

+----------+---------------+----------------+-------------+-------------+------------------+
|ID Cliente|MD_VLR_ESPORTES|MD_VLR_ALIMENTOS|MD_VLR_ROUPAS|MD_VLR_LIVROS|MD_VLR_ELETRONICOS|
+----------+---------------+----------------+-------------+-------------+------------------+
|       148|         105.19|          106.55|       139.53|         null|             63.83|
|       463|          63.06|            null|        80.97|       156.69|              null|
|       471|           null|           86.58|       150.03|         null|              null|
|       833|           null|            null|         null|       116.39|            175.14|
|       243|           null|            null|         null|         null|            157.53|
|       392|         191.38|          187.32|         null|       102.11|              null|
|       540|           53.6|            null|         null|        74.73|              null|
|       623|         109.34|            null|         null|         nu

### 5.3 Média do Valor Gasto nas categorias nos últimos 9 Meses:

In [56]:
df_valor_medio_9m = spark.sql("""
SELECT
    `ID Cliente`,
    ROUND(AVG(CASE WHEN Categoria = 'Esportes' THEN Valor ELSE NULL END), 2) AS MD_VLR_ESPORTES,
    ROUND(AVG(CASE WHEN Categoria = 'Alimentos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ALIMENTOS,
    ROUND(AVG(CASE WHEN Categoria = 'Roupas' THEN Valor ELSE NULL END), 2) AS MD_VLR_ROUPAS,
    ROUND(AVG(CASE WHEN Categoria = 'Livros' THEN Valor ELSE NULL END), 2) AS MD_VLR_LIVROS,
    ROUND(AVG(CASE WHEN Categoria = 'Eletrônicos' THEN Valor ELSE NULL END), 2) AS MD_VLR_ELETRONICOS
FROM
    df_transacoes
WHERE
    Data >= DATE_SUB(current_date(), 900)
GROUP BY
    `ID Cliente`
""")

df_valor_medio_9m.createOrReplaceTempView('df_valor_medio_9m')

In [57]:
df_valor_medio_9m.show()

+----------+---------------+----------------+-------------+-------------+------------------+
|ID Cliente|MD_VLR_ESPORTES|MD_VLR_ALIMENTOS|MD_VLR_ROUPAS|MD_VLR_LIVROS|MD_VLR_ELETRONICOS|
+----------+---------------+----------------+-------------+-------------+------------------+
|       148|          91.69|          106.55|       139.53|         null|             63.83|
|       463|          63.06|            null|        80.97|       156.69|            142.62|
|       471|           null|           86.58|       150.03|         null|            154.98|
|       833|           null|            null|         null|       116.39|            175.14|
|       243|           null|            null|         null|         null|            117.51|
|       392|         191.38|           99.92|       105.96|       102.11|              null|
|       540|           53.6|            null|         null|        74.73|            184.29|
|       623|         109.34|           70.97|         null|         nu

In [58]:
# Juntar todas as tabelas com variáveis explicativas na tabela final df_geral
df_geral = spark.sql("""
SELECT
    t.`ID Cliente`,
    t.Valor,
    t.Data,
    t.Categoria,
    u.ULTIMA_TRANSACAO_EM_DIAS,
    f.FREQ_COMPRA_MENSAL,
    tg.VALOR_TOTAL_GASTO,
    cf.Categoria_Favorita,
    gm.VALOR_TOTAL_POR_TRANS,
    i.DURACAO_ASSINATURA,
    vm3.MD_VLR_ESPORTES as MD_VLR_ESPORTES_3M,
    vm3.MD_VLR_ALIMENTOS as MD_VLR_ALIMENTOS_3M,
    vm3.MD_VLR_ROUPAS as MD_VLR_ROUPAS_3M,
    vm3.MD_VLR_LIVROS as MD_VLR_LIVROS_3M,
    vm3.MD_VLR_ELETRONICOS as MD_VLR_ELETRONICOS_3M,
    vm6.MD_VLR_ESPORTES as MD_VLR_ESPORTES_6M,
    vm6.MD_VLR_ALIMENTOS as MD_VLR_ALIMENTOS_6M,
    vm6.MD_VLR_ROUPAS as MD_VLR_ROUPAS_6M,
    vm6.MD_VLR_LIVROS as MD_VLR_LIVROS_6M,
    vm6.MD_VLR_ELETRONICOS as MD_VLR_ELETRONICOS_6M,
    vm9.MD_VLR_ESPORTES as MD_VLR_ESPORTES_9M,
    vm9.MD_VLR_ALIMENTOS as MD_VLR_ALIMENTOS_9M,
    vm9.MD_VLR_ROUPAS as MD_VLR_ROUPAS_9M,
    vm9.MD_VLR_LIVROS as MD_VLR_LIVROS_9M,
    vm9.MD_VLR_ELETRONICOS as MD_VLR_ELETRONICOS_9M
FROM df_transacoes t
LEFT JOIN df_ult_tras u ON t.`ID Cliente` = u.`ID Cliente`
LEFT JOIN df_freq_compra_mensal f ON t.`ID Cliente` = f.`ID Cliente`
LEFT JOIN df_total_gasto tg ON t.`ID Cliente` = tg.`ID Cliente`
LEFT JOIN df_favorita cf ON t.`ID Cliente` = cf.`ID Cliente`
LEFT JOIN df_gasto_medio_tras gm ON t.`ID Cliente` = gm.`ID Cliente`
LEFT JOIN df_intervalo i ON t.`ID Cliente` = i.`ID Cliente`
LEFT JOIN df_valor_medio_3m vm3 ON t.`ID Cliente` = vm3.`ID Cliente`
LEFT JOIN df_valor_medio_6m vm6 ON t.`ID Cliente` = vm6.`ID Cliente`
LEFT JOIN df_valor_medio_9m vm9 ON t.`ID Cliente` = vm9.`ID Cliente`
""")

df_geral.createOrReplaceTempView('df_geral')


In [59]:
df_geral.show()

+----------+------------------+--------------------+-----------+------------------------+------------------+-----------------+------------------+---------------------+------------------+------------------+-------------------+----------------+----------------+---------------------+------------------+-------------------+----------------+----------------+---------------------+------------------+-------------------+----------------+----------------+---------------------+
|ID Cliente|             Valor|                Data|  Categoria|ULTIMA_TRANSACAO_EM_DIAS|FREQ_COMPRA_MENSAL|VALOR_TOTAL_GASTO|Categoria_Favorita|VALOR_TOTAL_POR_TRANS|DURACAO_ASSINATURA|MD_VLR_ESPORTES_3M|MD_VLR_ALIMENTOS_3M|MD_VLR_ROUPAS_3M|MD_VLR_LIVROS_3M|MD_VLR_ELETRONICOS_3M|MD_VLR_ESPORTES_6M|MD_VLR_ALIMENTOS_6M|MD_VLR_ROUPAS_6M|MD_VLR_LIVROS_6M|MD_VLR_ELETRONICOS_6M|MD_VLR_ESPORTES_9M|MD_VLR_ALIMENTOS_9M|MD_VLR_ROUPAS_9M|MD_VLR_LIVROS_9M|MD_VLR_ELETRONICOS_9M|
+----------+------------------+--------------------+----